# Pacvolt Prototyping Cookbook


Experimental work for Pacvolt

%matplotlib inline

In [15]:
# import general libraries
import os
import sys
print(sys.version)

#import numpy as np
#import pandas as pd
#import plotly.offline as pyo
# import matplotlib as mpl
# import requests

# install dtat from dtat-library
# https://github.jpl.nasa.gov/Europa-PESS/dtat-library

import pandas as pd
from copy import copy

import dtat
import dtat.plot as dtatplot
from dtat.dataconnectors.csvconnector import CSVConnector
from dtat.types import CustomizedTrace

import plotly.io as pio

import numpy as np

# filter out warning messages; comment this when debugging
import warnings
warnings.filterwarnings('ignore')

# import FPSA libraries
#import dtat.stacked as dtat

%matplotlib inline

# Dev venue primary collection name
COLLECTION_NAME = "STATE_MANAGER_DEMO"

import dtat.demo_data as fake_data


3.13.7 (main, Aug 14 2025, 11:12:11) [Clang 17.0.0 (clang-1700.0.13.3)]


In [13]:
# Plot some data

# faults = CSVConnector("~/dev_pacvolt/pacvolt-analysis/faults.csv").get_data()

pio.renderers.default = 'iframe'


#faults2 = [("time","name","value","unit"),
#    ("2025-12-12 15:00:00","Fault","1","none"),
#         ("2025-12-12 18:00:00","Fault","1","none"),
#         ("2025-12-13 00:00:00","Fault","1","none")]

#faults = [{"scet":"2025-12-13 15:00:00", "name": "Fault", "value": 1, "unit": "none"},
#          {"scet":"2025-12-12 18:00:00", "name": "Fault", "value": 1, "unit": "none"},
#          {"scet":"2025-12-13 00:00:00", "name": "Fault", "value": 1, "unit": "none"}]

# faults_df = pd.DataFrame(faults)

#data_for_set = CSVConnector("/Users/gameraman/dev/dev_pacvolt/pacvolt-analysis/data.csv").get_data()
data_for_set = CSVConnector("/tmp/pva_test_output.csv").get_data()

# print(data_for_set)

# df = pd.concat([faults_df, data_for_set], ignore_index=True)

## all_data = [pd.concat([data_for_set, faults], ignore_index=True)]


fig,c,m,t = dtatplot.make_stacked_graph( # make_stacked_graph is the primary plotting method
    data_for_set,  # this is the data that DTAT plots from
    # y_vars = [["PWR_BUS_VOLTAGE_1"],["PWR_VOLTAGE_1"],["PWR_BATTERY_VOLTAGE_1"],["PWR_CURRENT_1"]], # this is the y variables to plot
    x_var = "scet",
    y_vars = [["AvgVin","AvgVout"],["AvgAmps"]],
    customize_dict={ # by passing the CustomizedTrace in a dictionary with y variables as the keys, you can control more trace settings
        'AvgVin': CustomizedTrace(
                                    #size=10,  # setting the marker size
                                    #symbol='triangle-down-open',  # setting the marker shape, any plotly option is valid
                                    color='cadetblue',  # setting the marker color
                                ),
                                'Fault': CustomizedTrace(
                                    size=12,  # setting the marker size
                                    symbol='diamond',  # setting the marker shape, any plotly option is valid
                                    color='red',  # setting the marker color
                                )
                             
                             }

                             
                             
    )  # the x variable to plot by defaults to SCET
fig.show()




In [84]:
# Sub-sample voltage data based on time windows

# Get the full dataset
data_turn_on = CSVConnector("/Users/gameraman/dev/dev_pacvolt/pacvolt-analysis/data.csv").get_data()

data_turn_on['scet'] = pd.to_datetime(data_turn_on['scet'])

# Filter to only Fault data
faults = data_turn_on[data_turn_on['name'] == 'Fault'].copy()

print (faults)
# fault_list = faults.to_numpy()

 
# Filter to only Voltage data
# voltage_data = data_turn_on[data_turn_on['name'] == 'Voltage'].copy()


# Convert scet column to datetime
# voltage_data['scet'] = pd.to_datetime(data_turn_on['scet'])

# Define the time range in minutes
range_minutes = 5

# Loop through each time in times array

time_list = faults['scet'].tolist()


object_array = faults.to_numpy()

for time in time_list:
    # Convert time to pandas Timestamp and remove timezone info to match data_turn_on
    old_center_time = pd.Timestamp(time).tz_convert("UTC")
    # center_time = old_center_time + pd.Timedelta(hours=9)

    print ("time")
    print (center_time)
    print ("old_center")
    print(old_center_time)
    

    # Calculate time window
    time_min = center_time - pd.Timedelta(minutes=range_minutes)
    time_max = center_time + pd.Timedelta(minutes=range_minutes)

    print ("min")
    print(time_min)
    print("max")
    print(time_max)
    
    # Filter data within the range
    sub_sample = data_turn_on[(data_turn_on['scet'] >= time_min) & (data_turn_on['scet'] <= time_max)]
    
    
    # faults_sample = faults[(faults['scet'] >= time_min) & (faults['scet'] <= time_max)]
    

    #print("faults_sample")
    # print(faults_sample)
 

    print (sub_sample)
    
    # print(f"\nSub-sample for time {time} (±{range_minutes} minutes):")
    # print(sub_sample)
    # print(f"Number of rows: {len(sub_sample)}")
    fig,c,m,t = dtatplot.make_stacked_graph( # make_stacked_graph is the primary plotting method
    sub_sample,  # this is the data that DTAT plots from
    # y_vars = [["PWR_BUS_VOLTAGE_1"],["PWR_VOLTAGE_1"],["PWR_BATTERY_VOLTAGE_1"],["PWR_CURRENT_1"]], # this is the y variables to plot
    x_var = "scet",
    y_vars = [["Voltage"],["Fault"]],
    customize_dict={ # by passing the CustomizedTrace in a dictionary with y variables as the keys, you can control more trace settings
        'Voltage': CustomizedTrace(
                                    #size=10,  # setting the marker size
                                    #symbol='triangle-down-open',  # setting the marker shape, any plotly option is valid
                                    color='cadetblue',  # setting the marker color
                                ),
                                'Fault': CustomizedTrace(
                                    size=12,  # setting the marker size
                                    symbol='diamond',  # setting the marker shape, any plotly option is valid
                                    color='red',  # setting the marker color
                                )
                             
                             }

                             
                             
    )


    fig.update_layout(
    shapes=[
        dict(
            type="line",
            xref="x",
            yref="paper", # 'paper' makes the line span the entire y-axis height (0 to 1)
            x0=center_time,
            y0=0,
            x1=center_time,
            y1=1,
            line=dict(
                color="black",
                width=1,
                dash="dash"
            )
        )
    ]
)


    
    # the x variable to plot by defaults to SCET
    fig.show()

    print 


                       scet   name value  unit
0 2025-12-15 09:00:00+00:00  Fault     1  none
1 2025-12-15 10:00:00+00:00  Fault     1  none
2 2025-12-15 12:00:00+00:00  Fault     1  none
time
2025-12-15 21:00:00+00:00
old_center
2025-12-15 09:00:00+00:00
min
2025-12-15 20:55:00+00:00
max
2025-12-15 21:05:00+00:00
                          scet     name               value   unit
720  2025-12-15 20:55:00+00:00  Voltage  29.099290879458138  volts
721  2025-12-15 20:56:00+00:00  Voltage  27.153155282745622  volts
722  2025-12-15 20:57:00+00:00  Voltage  23.465482833299635  volts
723  2025-12-15 20:58:00+00:00  Voltage  29.864300447029734  volts
724  2025-12-15 20:59:00+00:00  Voltage   27.74914175439936  volts
725  2025-12-15 21:00:00+00:00  Voltage  22.939301273413914  volts
726  2025-12-15 21:01:00+00:00  Voltage  23.399205506527885  volts
727  2025-12-15 21:02:00+00:00  Voltage   29.05544488692035  volts
728  2025-12-15 21:03:00+00:00  Voltage   26.46590972252391  volts
729  2025-12-1

time
2025-12-15 21:00:00+00:00
old_center
2025-12-15 10:00:00+00:00
min
2025-12-15 20:55:00+00:00
max
2025-12-15 21:05:00+00:00
                          scet     name               value   unit
720  2025-12-15 20:55:00+00:00  Voltage  29.099290879458138  volts
721  2025-12-15 20:56:00+00:00  Voltage  27.153155282745622  volts
722  2025-12-15 20:57:00+00:00  Voltage  23.465482833299635  volts
723  2025-12-15 20:58:00+00:00  Voltage  29.864300447029734  volts
724  2025-12-15 20:59:00+00:00  Voltage   27.74914175439936  volts
725  2025-12-15 21:00:00+00:00  Voltage  22.939301273413914  volts
726  2025-12-15 21:01:00+00:00  Voltage  23.399205506527885  volts
727  2025-12-15 21:02:00+00:00  Voltage   29.05544488692035  volts
728  2025-12-15 21:03:00+00:00  Voltage   26.46590972252391  volts
729  2025-12-15 21:04:00+00:00  Voltage  27.729825809238115  volts
730  2025-12-15 21:05:00+00:00  Voltage  23.550962258894504  volts
1720 2025-12-15 20:55:00+00:00  Current   5.638250921513667   amps
1

time
2025-12-15 21:00:00+00:00
old_center
2025-12-15 12:00:00+00:00
min
2025-12-15 20:55:00+00:00
max
2025-12-15 21:05:00+00:00
                          scet     name               value   unit
720  2025-12-15 20:55:00+00:00  Voltage  29.099290879458138  volts
721  2025-12-15 20:56:00+00:00  Voltage  27.153155282745622  volts
722  2025-12-15 20:57:00+00:00  Voltage  23.465482833299635  volts
723  2025-12-15 20:58:00+00:00  Voltage  29.864300447029734  volts
724  2025-12-15 20:59:00+00:00  Voltage   27.74914175439936  volts
725  2025-12-15 21:00:00+00:00  Voltage  22.939301273413914  volts
726  2025-12-15 21:01:00+00:00  Voltage  23.399205506527885  volts
727  2025-12-15 21:02:00+00:00  Voltage   29.05544488692035  volts
728  2025-12-15 21:03:00+00:00  Voltage   26.46590972252391  volts
729  2025-12-15 21:04:00+00:00  Voltage  27.729825809238115  volts
730  2025-12-15 21:05:00+00:00  Voltage  23.550962258894504  volts
1720 2025-12-15 20:55:00+00:00  Current   5.638250921513667   amps
1

In [64]:
# Read data from fake_data.instrument_turn_on() and save to CSV
import csv

data = fake_data.instrument_turn_on()

# Convert scet column to the format YYYY-DDDTHH:MM:SS (Julian day format)
data['scet'] = pd.to_datetime(data['scet']).dt.strftime('%Y-%jT%H:%M:%S')

# Save to CSV in the same format as faults.csv with quotes around non-numeric columns
output_path = os.path.expanduser("~/dev_pacvolt/pacvolt-analysis/data.csv")
data.to_csv(output_path, index=False, quoting=csv.QUOTE_NONNUMERIC)

print(f"Data saved to: {output_path}")
print(f"Number of rows: {len(data)}")
print("\nFirst few rows:")
print(data.head())

OSError: Cannot save file into a non-existent directory: '/Users/gameraman/dev_pacvolt/pacvolt-analysis'

In [ ]:
# Convert and save notebook as HTML
import os
import nbformat
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor

# Define output directory
output_dir = "/Users/gameraman/dev_pacvolt/output"
os.makedirs(output_dir, exist_ok=True)

# Read the current notebook
notebook_path = "/Users/gameraman/dev_pacvolt/pacvolt-analysis/DTAT_CTU.ipynb"
with open(notebook_path, 'r') as f:
    nb = nbformat.read(f, as_version=4)

# Execute the notebook
ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
ep.preprocess(nb)

# Convert to HTML
html_exporter = HTMLExporter()
html_exporter.template_name = 'classic'  # Use classic template for better formatting
(body, resources) = html_exporter.from_notebook_node(nb)

# Save HTML file
output_html = os.path.join(output_dir, "output.html")
with open(output_html, 'w', encoding='utf-8') as f:
    f.write(body)

# Save any additional resources (images, etc.)
for filename, data in resources.get('outputs', {}).items():
    output_file = os.path.join(output_dir, filename)
    with open(output_file, 'wb') as f:
        f.write(data)

print(f"Notebook saved as HTML to: {output_html}")
print(f"Additional files saved to: {output_dir}")

In [12]:
import csv

data = fake_data.drifting_off_nominal()

# Convert scet column to the format YYYY-DDDTHH:MM:SS (Julian day format)
data['scet'] = pd.to_datetime(data['scet']).dt.strftime('%Y-%jT%H:%M:%S')

# Save to CSV in the same format as faults.csv with quotes around non-numeric columns
output_path = os.path.expanduser("/Users/gameraman/dev/dev_pacvolt/pacvolt-analysis/drift.csv")
data.to_csv(output_path, index=False, quoting=csv.QUOTE_NONNUMERIC)

print(f"Data saved to: {output_path}")
print(f"Number of rows: {len(data)}")
print("\nFirst few rows:")
print(data.head())

Data saved to: /Users/gameraman/dev/dev_pacvolt/pacvolt-analysis/drift.csv
Number of rows: 3000

First few rows:
                scet     name      value
0  2025-349T18:26:00  Voltage  45.476553
1  2025-349T18:27:00  Voltage  48.065325
2  2025-349T18:28:00  Voltage  53.506392
3  2025-349T18:29:00  Voltage  51.300148
4  2025-349T18:30:00  Voltage  54.859507


In [23]:
# Plot some data

# faults = CSVConnector("~/dev_pacvolt/pacvolt-analysis/faults.csv").get_data()

pio.renderers.default = 'iframe'


#faults2 = [("time","name","value","unit"),
#    ("2025-12-12 15:00:00","Fault","1","none"),
#         ("2025-12-12 18:00:00","Fault","1","none"),
#         ("2025-12-13 00:00:00","Fault","1","none")]

#faults = [{"scet":"2025-12-13 15:00:00", "name": "Fault", "value": 1, "unit": "none"},
#          {"scet":"2025-12-12 18:00:00", "name": "Fault", "value": 1, "unit": "none"},
#          {"scet":"2025-12-13 00:00:00", "name": "Fault", "value": 1, "unit": "none"}]

# faults_df = pd.DataFrame(faults)

#data_for_set = CSVConnector("/Users/gameraman/dev/dev_pacvolt/pacvolt-analysis/data.csv").get_data()
data_for_set = CSVConnector("/tmp/pva_test_output.csv").get_data()

# print(data_for_set)

# df = pd.concat([faults_df, data_for_set], ignore_index=True)

## all_data = [pd.concat([data_for_set, faults], ignore_index=True)]


fig,c,m,t = dtatplot.make_stacked_graph( # make_stacked_graph is the primary plotting method
    data_for_set,  # this is the data that DTAT plots from
    # y_vars = [["PWR_BUS_VOLTAGE_1"],["PWR_VOLTAGE_1"],["PWR_BATTERY_VOLTAGE_1"],["PWR_CURRENT_1"]], # this is the y variables to plot
    x_var = "scet",
    y_vars = [["AvgVin","AvgVout"],["AvgAmps"],["Avg_kVA"],["Avg_kW"]],
    customize_dict={ # by passing the CustomizedTrace in a dictionary with y variables as the keys, you can control more trace settings
        'AvgVin': CustomizedTrace(
                                    #size=10,  # setting the marker size
                                    #symbol='triangle-down-open',  # setting the marker shape, any plotly option is valid
                                    color='red',  # setting the marker color
                                ),
                                'Fault': CustomizedTrace(
                                    size=12,  # setting the marker size
                                    symbol='diamond',  # setting the marker shape, any plotly option is valid
                                    color='red',  # setting the marker color
                                )
                             
                             }

                             
                             
    )  # the x variable to plot by defaults to SCET
fig.show()


